In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial import distance
from ipywidgets import AppLayout, Button, Select, Layout, jslink, IntText, Text, IntSlider, HTML, Dropdown, Output, Textarea, VBox, Label
from IPython.display import display, clear_output
from sklearn.metrics.pairwise import cosine_similarity

#
# Initialization
#

# Read in clustered data
df = pd.read_csv("music_genre_cleaned_kmeans")

# Joining artist and track name from main DataFrame into a Series
ds_artist_track_name = df[['artist_name']].sort_values(by=['artist_name'])['artist_name'] + ' - ' + df[['artist_name', 'track_name']].sort_values(by=['artist_name', 'track_name'])['track_name']


#
# Layout
#

# Styling
h1 = dict(font_size='30px', font_family='sans-serif', font_weight='bold')
p = dict(font_size='20px', font_family='sans-serif', font_weight='bold')
auto_layout = Layout(width='auto', height='auto')
button_style = Layout(width='50%', height='50px', visibility='visible')

# Elements
header_label = Label('Music Playlist Generator', style=h1)

search_label = Label('Search', style=p)
search_box = Text(value='', layout=Layout(width='50%', height='50px')) # User input search box - string is stored in search_box.value
search_button = Button(description='Search', layout=button_style) # Search button 
song_selection = Select(description='', layout=Layout(visibility='hidden'))
generate_button = Button(description='Generate', layout=Layout(visibility='hidden', width='50%', height='50px'))
results_vbox = VBox([song_selection]) # Box displaying results

# Vbox
header_vbox = VBox([header_label], layout=Layout(width='100%',
                                                 height='100%',
                                                 align_items='center'))

center_vbox = VBox([search_label,
                  search_box,
                  search_button,
                  song_selection,
                  generate_button], layout=Layout(width='100%',
                                                  height='100%',
                                                  align_items='center'))


# 
# Functions
#

def song_search(user_input):
    """ Song Search Function
    Searches for a user input string in ds_artist_track_name and returns any matches as a Series
    Atrributes:
        search_results: A dictionary containing any matching values mapped to its index
        
    :param user_input: A user input string provided by search_box.value
    """  
    if user_input.lower().replace('-', '').replace(' ', '') == '':
        return None
    search_results = {}
    for i in ds_artist_track_name.index:
        if user_input.lower().replace('-', '').replace(' ', '') in ds_artist_track_name.iat[i].lower().replace('-', '').replace(' ', ''):
            search_results.update({ds_artist_track_name.iat[i] : i})
            if len(search_results) > 50:
                break
    song_selection.options = search_results
    song_selection.layout=Layout(width='80%', height='100%', visiblity='visible')
    generate_button.layout=button_style
    
def generate_playlist(song_id):
    """ Generate Playlist Function
    Generates a playlist given a user selected item in the song_selection box
    """
    # Create a subset from df where kmeans_20 is equal to that of the selected song with only transformed data
    df_subset_T = df[df['kmeans_20'] == df.iloc[song_selection.value].loc['kmeans_20']] [['acousticness_T', 'danceability_T', 'energy_T', 'instrumentalness_T',
                      'liveness_T', 'loudness_T', 'speechiness_T', 'tempo_T', 'valence_T']] 
    
    # Generate a similarity score between the selected song and df_subset_T using cosine_similarity
    similarity_score = []
    for i in df_subset_T.index:
        similarity_score.append(cosine_similarity(df_subset_T.loc[[song_selection.value]],  df_subset_T.loc[[i]])[0][0])
    
    # Create the output DataFrame assign the similarity score to it
    df_subset_T = df[df['kmeans_20'] == df.iloc[song_selection.value].loc['kmeans_20']] [['artist_name', 'track_name']]
    df_subset_T = df_subset_T.assign(similarity_score=similarity_score)
    
    # Filter the output DataFrame by genre with the top 30 similar songs
    song_genre = df.iloc[song_id].loc['music_genre'] # used to filter by genre
    recommendation_playlist = df_subset_T.loc[(df['music_genre'] == song_genre)].sort_values(by=['similarity_score'], ascending=False).head(30)[['artist_name', 'track_name', 'similarity_score']]
    with output:
        clear_output()
        display(recommendation_playlist.rename(columns={'artist_name': 'Artist', 'track_name': 'Track', 'similarity_score': 'Similarity Score'}).style.hide(axis="index"))
    return output

def search_button_clicked(b):
    """ Search Button Clicked
    Runs the song_search function
    """
    with output:
        song_search(search_box.value)

def generate_button_clicked(b):
    """ Generate Button Clicked
    Runs the generate_playlist function
    """
    with output:
        generate_playlist(song_selection.value)
    return


#
# Main
#
output = Output()
display(output)
search_button.on_click(search_button_clicked)
generate_button.on_click(generate_button_clicked)

AppLayout(header=header_vbox,
          left_sidebar=None,
          center=center_vbox,
          right_sidebar=None,
          footer=None, 
          pane_heights=['50px', '600px', '100px'],
          pane_widths=[4, 6, 0],)

Output()

AppLayout(children=(VBox(children=(Label(value='Music Playlist Generator', style=LabelStyle(font_family='sans-…